# Q-learning first attempt

Optionally install what you need. 

In [1]:
import sys
!{sys.executable} -m pip install seaborn gym-retro Pillow keras tensorflow opencv-python pandas matplotlib scipy

First we set up all of the imports that we will have with the project

In [2]:
import random
import math
import retro
from PIL import Image
import gym
import pickle
import operator
import pprint
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import cv2 #OpenCV
import time
import gym_remote.client as grc
import gym_remote.exceptions as gre
import os
import json
import pandas as pd
from IPython.display import clear_output
from collections import deque
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (30, 30)
import seaborn as sns

from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf


/Users/tristansokol/Development/Bobcats/reverie_agent/reverie_agent/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Hyperparameters

Set up what a tracked environment is

In [3]:
def proccessObs(obs):
    window_size_x = 180
    window_size_y = 100
    window_offset_x = 140 #int((320 - window_size_x)/2)
    window_offset_y = 100 #int((224 - window_size_y)/2)
    # lm = Image.fromarray(np.array(obs[window_offset_y:(window_offset_y+window_size_y),window_offset_x:(window_offset_x+window_size_x)]))
    # lm.show()
    # input("Press Enter to continue...")
    return obs[window_offset_y:(window_offset_y+window_size_y),window_offset_x:(window_offset_x+window_size_x)].flatten().tostring()



In [4]:
# file system interactors
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        if name == 'epsilon':
            return .7;
        return []
    else:
        return []

In [5]:
def move(env, num_steps, left=False, jump_prob=1.0 / 10.0, jump_repeat=4):
    """
    Move right or left for a certain number of steps,
    jumping periodically.
    """
    total_rew = 0.0
    done = False
    steps_taken = 0
    jumping_steps_left = 0
    while not done and steps_taken < num_steps:

        #["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
        action = np.zeros((12,), dtype=np.bool)
        action[6] = left
        action[7] = not left
        if jumping_steps_left > 0:
            action[0] = True
            jumping_steps_left -= 1
        else:
            if random.random() < jump_prob:
                jumping_steps_left = jump_repeat - 1
                action[0] = True
        # no info variable in contest variable
        obs, rew, done, _ = env.step(action)
        total_rew += rew
        steps_taken += 1
        if done:
            break
    return total_rew, done

In [6]:
def exploit(env, sequence):
    """
    Replay an action sequence; pad with NOPs if needed.

    Returns the final cumulative reward.
    """
    env.reset()
    done = False
    idx = 0
    while not done:
        if idx >= len(sequence):
            # _, _, done, _ = env.step(np.zeros((12,), dtype='bool'))
            _, done = move(env, 5, jump_repeat=1)
        else:
            _, _, done, _ = env.step(sequence[idx])
        idx += 1
    return env.total_reward

In [7]:
output_dir = './run-'+time.strftime("%Y%m%d-%H:%M")+'/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
loss_file_path = output_dir+"loss_df.csv"

#Intialize log structures from file if exists else create new#Intiali 
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])


In [8]:
class TrackedEnv(gym.Wrapper):
    """
    An environment that tracks the current trajectory and
    the total number of timesteps ever taken.
    """

    def __init__(self, env):
        super(TrackedEnv, self).__init__(env)
        self.action_history = []
        self.reward_history = []
        self.total_reward = 0
        self.total_steps_ever = 0
        record_file_path = output_dir+"record.csv"
        self.record = pd.read_csv(record_file_path) if os.path.isfile(record_file_path) else pd.DataFrame(columns = ['Timesteps','Total_Score'])
        actions_file_path = output_dir+"actions.csv"
        self.actions = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['Action','Intention'])
        
    def best_sequence(self):
        """
        Get the prefix of the trajectory with the best
        cumulative reward.
        """
        max_cumulative = max(self.reward_history)
        for i, rew in enumerate(self.reward_history):
            if rew == max_cumulative:
                return self.action_history[:i + 1]
        raise RuntimeError('unreachable')

    # pylint: disable=E0202
    def reset(self, **kwargs):
        self.action_history = []
        self.reward_history = []
        self.total_reward = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        self.total_steps_ever += 1
        self.action_history.append(action.copy())
        obs, rew, done, info = self.env.step(action)
        if done:
            data = pd.DataFrame({'Timesteps':[len(self.action_history)], 'Total_Score': [self.total_reward]})
            self.record = self.record.append(data)
#             self.record.loc[len(loss_df)] = score
#             self.record.append([self.total_reward, len(self.action_history)])
            print('rip')
        self.total_reward += rew
        self.reward_history.append(self.total_reward)
        return obs, rew, done, info


In [9]:
EXPLOIT_BIAS = 0.25  # 0.;5
TOTAL_TIMESTEPS = int(1e6)

#game parameters
ACTIONS = 8 # possible actions
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 200000. # timesteps to observe before training
EXPLORE = 200000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 100000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAMERATE= 4 #how often to render
LEARNING_RATE = 1e-4
img_rows , img_cols = 120,84
img_channels = 4 #We stack 4 frames

In [10]:
def move_n_learn(env, num_steps, left=False, jump_prob=1.0 / 10.0, jump_repeat=4):
    """
    Move right or left for a certain number of steps,
    jumping periodically.
    """
    total_rew = 0.0
    done = False
    steps_taken = 0
    jumping_steps_left = 0
    while not done and steps_taken < num_steps:

        #["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
        action = np.zeros((12,), dtype=np.bool)
        action[6] = left
        action[7] = not left
        if jumping_steps_left > 0:
            action[0] = True
            jumping_steps_left -= 1
        else:
            if random.random() < jump_prob:
                jumping_steps_left = jump_repeat - 1
                action[0] = True
        # no info variable in contest variable
        obs, rew, done, _ = env.step(action)

        total_rew += rew
        steps_taken += 1
        if done:
            
            break
    return total_rew, done

In [11]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=(img_cols,img_rows,img_channels)))  #20*40*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mean_squared_logarithmic_error',optimizer=adam)
    print("We finish building the model")
    return model
# buildmodel().summary()

In [12]:
def random_move(env, Choice=None):
    done = False
    if Choice is None:
        Choice = random.randint(1,len(actions))-1
    # no info variable in contest environment
    obs, rew, done, _ = env.step(actions[Choice])
    
    return rew, done, obs, Choice


In [13]:
# Possible actions
# ["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
actions = [
    [True, False, False, False, False, False, False, False, False, False, False, False],
    [True, False, False, False, False, False, False, True, False, False, False, False],
    [False, False, False, False, False, False, False, True, False, False, False, False],
    [False, False, False, False, False, True, False, False, False, False, False, False],
    [True, False, False, False, False, True, False, False, False, False, False, False],
    [False, False, False, False, False, False, False, False, False, False, False, False],
    [False, False, False, False, False, False, True, False, False, False, False, False],
    [True, False, False, False, False, False, True, False, False, False, False, False],
]

In [14]:
def process_img(image):
    #crop out the dino agent from the frame
    height, width = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     print(height)
#     print(width)
    image = image[round(height*.25):height,round(width*.25):width] #img[y:y+h, x:x+w] 
    image = cv2.resize(image, (0,0), fx = 0.5, fy = 0.5) 
#     height, width = image.shape[:2]
#     print(height)
#     print(width)
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200) #apply the canny edge detection
    return  image   

In [15]:
def show_img(image,graphs = False):
    """
    Show images in new window
    """
    while True:
#         print(image.shape)
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        processed = process_img(image)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)  
        cv2.moveWindow(window_title, 20,20);
#         imS = cv2.resize(screen, (800, 400)) 
#         cv2.imshow(window_title, screen)
        cv2.imshow(window_title, processed)
        cv2.waitKey(5)
        cv2.destroyAllWindows
        break


In [16]:
def main(observe=False):
    # Set up a new TrackedEnv that can keep track of total timestamps and store
    # previous best solutions.
    #
    # env = grc.RemoteEnv('tmp/sock')
    # env = TrackedEnv(env)

    
    env = retro.make(game='SonicTheHedgehog-Genesis',
                     state='GreenHillZone.Act1',
                     scenario='contest',
                     record=output_dir)
    env = TrackedEnv(env)

    # new_ep will keep track of if a new episode should be started.
    new_ep = True
    # solutions is an array of successful gameplay sequences as well as the
    
    solutions = []
    
    model = buildmodel()
    x_t = process_img(np.zeros((224,320,3), dtype=np.uint8))
    
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4

    initial_state = s_t
    if observe :
        OBSERVE = 999999999    #We keep observing, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
#         model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        
    t = 0 # start of timesteps
    D = deque()
    while True:
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        if new_ep:
            clear_output(wait=True)
            print('%f%% done, reward: %f' % (env.total_steps_ever / 10000, env.record["Total_Score"].mean()))
            
#             if (solutions and
#                     random.random() < EXPLOIT_BIAS + env.total_steps_ever / TOTAL_TIMESTEPS):
#                 solutions = sorted(solutions, key=lambda x: np.mean(x[0]))
#                 best_pair = solutions[-1]
#                 new_rew = exploit(env, best_pair[1])
#                 best_pair[0].append(new_rew)
#                 print('replayed best with reward %f' % new_rew)
#                 continue
#             else:
            env.reset()
            new_ep = False
        if  random.random() <= epsilon: #randomly explore an action
#             print("----------Random Action----------")
            action_index = random.randrange(len(actions[:]))
            env.actions.loc[len(env.actions)]= {'Action':action_index, 'Intention': 'Random'}

           
        else: # predict the output
#             print("----------Predicted----------")
            q = model.predict(s_t)       #input a stack of 4 images, get the prediction
            max_Q = np.argmax(q)         # chosing index with maximum q value
            action_index = max_Q 
            env.actions.loc[len(env.actions)]= {'Action':action_index, 'Intention': 'Predicted'}
   
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON  and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE         
                      
#         action_index=2;
#     x_t1 ~ obs? terminal ~ done
        #run the selected action and observed next state and reward
#         x_t1, r_t, terminal = game_state.get_state(a_t)
#         print(t,'doing action',action_index)
        reward, done, obs, choice = random_move(env,Choice=action_index)
        x_t1 = process_img(obs)
#         show_img(obs)
#         if t % FRAMERATE ==0:
#             env.render()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x168x320x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        D.append((s_t, action_index, reward, s_t1, done))

        if len(D) > REPLAY_MEMORY:
            D.popleft()
            
               #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on

            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
        s_t = initial_state if done else s_t1 #reset game to initial frame if terminate
        t = t + 1
#         env.render()
#         print(t, env.total_reward )
#         print(env.record)
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"
        if t % 100 == 0:
            print("T", t, "/ STATE", state,"/ ε", round(epsilon,3), "/ REWARD", round(env.total_reward),"/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

        if done:
            new_ep = True
        
        if t % 1000 == 0:
            model.save_weights(output_dir+"model_weights.h5", overwrite=True)
#             save_obj(D,output_dir+"D") #saving episodes
            save_obj(t,output_dir+"time") #caching time steps
            save_obj(epsilon,output_dir+"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv(output_dir+"loss_df.csv",index=False)
            env.record.to_csv(output_dir+"records.csv",index=False)
            env.actions.to_csv(output_dir+"actions.csv",index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
        if t >1000000:
            exit();
# #         rew, new_ep = move_n_learn(env, 1)
#         if not new_ep and rew <= 0:
#             print('backtracking due to negative reward: %f' % rew)
#             _, new_ep = move_n_learn(env, 70, left=True)
#         if new_ep:
#             solutions.append(([max(env.reward_history)], env.best_sequence()))

In [17]:
def show_plots():
    rundir = 'run-20180519-11:12'
    fig, axs = plt.subplots(ncols=1,nrows =3,figsize=(15,15))
    axs[0].set_title('Loss')
    axs[1].set_title('Game Score progress')
    loss_df = pd.read_csv("./"+rundir+"/loss_df.csv").tail(100000) #.set_yscale('log')
    scores_df = pd.read_csv("./"+rundir+"/records.csv").head(190000)
    actions_df = pd.read_csv("./"+rundir+"/actions.csv").tail(100000)
    actions_df['Action'] = actions_df['Action'].astype('float') 
    loss_df['loss'] = loss_df['loss'].astype('float') 
    loss_df.plot(use_index=True,ax=axs[0]).set_yscale('log')


    scores_df.plot(ax=axs[1])
    sns.distplot(actions_df['Action'].loc[actions_df['Intention'] == 'Predicted'])
    sns.distplot(actions_df['Action'].loc[actions_df['Intention'] == 'Random'])
    imgg = fig.canvas.draw()
# show_plots()

In [ ]:
if __name__ == '__main__':
    try:
        main()
    except gre.GymRemoteError as exc:
        print('exception', exc)


34.749200% done, reward: 467.815808
T 347493 / STATE explore / ε 0.626 / ACTION 6 / REWARD 0 / Q_MAX  1241731.8 / Loss  87385216.0
T 347494 / STATE explore / ε 0.626 / ACTION 7 / REWARD 0 / Q_MAX  1097242.5 / Loss  58908628.0
T 347495 / STATE explore / ε 0.626 / ACTION 7 / REWARD 0 / Q_MAX  1126298.8 / Loss  73176392.0
T 347496 / STATE explore / ε 0.626 / ACTION 4 / REWARD 0 / Q_MAX  908341.94 / Loss  64433628.0
T 347497 / STATE explore / ε 0.626 / ACTION 1 / REWARD 0 / Q_MAX  1156137.2 / Loss  54869552.0
T 347498 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1061583.4 / Loss  101153000.0
T 347499 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1161758.4 / Loss  53842196.0
T 347500 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1047353.75 / Loss  123531520.0
T 347501 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1130527.1 / Loss  77976928.0
T 347502 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1111923.1 / Loss  80979312.0
T 347503 / 

T 347579 / STATE explore / ε 0.626 / ACTION 2 / REWARD 0 / Q_MAX  906302.2 / Loss  43854100.0
T 347580 / STATE explore / ε 0.626 / ACTION 1 / REWARD 0 / Q_MAX  1074359.1 / Loss  40551916.0
T 347581 / STATE explore / ε 0.626 / ACTION 1 / REWARD 0 / Q_MAX  1075701.1 / Loss  48676304.0
T 347582 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1078261.9 / Loss  95350896.0
T 347583 / STATE explore / ε 0.626 / ACTION 1 / REWARD 1 / Q_MAX  908474.7 / Loss  50001900.0
T 347584 / STATE explore / ε 0.626 / ACTION 5 / REWARD 2 / Q_MAX  1059194.1 / Loss  54155800.0
T 347585 / STATE explore / ε 0.626 / ACTION 5 / REWARD 3 / Q_MAX  1041140.44 / Loss  71247120.0
T 347586 / STATE explore / ε 0.626 / ACTION 2 / REWARD 4 / Q_MAX  1086706.8 / Loss  80514320.0
T 347587 / STATE explore / ε 0.626 / ACTION 5 / REWARD 5 / Q_MAX  1041648.44 / Loss  70189344.0
T 347588 / STATE explore / ε 0.626 / ACTION 3 / REWARD 6 / Q_MAX  1061258.8 / Loss  55911352.0
T 347589 / STATE explore / ε 0.626 / ACTION 7 / RE

T 347665 / STATE explore / ε 0.626 / ACTION 1 / REWARD 27 / Q_MAX  1055711.0 / Loss  61832364.0
T 347666 / STATE explore / ε 0.626 / ACTION 5 / REWARD 27 / Q_MAX  1089319.1 / Loss  70707032.0
T 347667 / STATE explore / ε 0.626 / ACTION 5 / REWARD 27 / Q_MAX  827119.9 / Loss  85740440.0
T 347668 / STATE explore / ε 0.626 / ACTION 6 / REWARD 27 / Q_MAX  919584.25 / Loss  19996892.0
T 347669 / STATE explore / ε 0.626 / ACTION 6 / REWARD 27 / Q_MAX  1073696.1 / Loss  26288442.0
T 347670 / STATE explore / ε 0.626 / ACTION 5 / REWARD 27 / Q_MAX  1114447.0 / Loss  135791008.0
T 347671 / STATE explore / ε 0.626 / ACTION 5 / REWARD 26 / Q_MAX  1090021.1 / Loss  67494176.0
T 347672 / STATE explore / ε 0.626 / ACTION 3 / REWARD 26 / Q_MAX  1026495.7 / Loss  45577008.0
T 347673 / STATE explore / ε 0.626 / ACTION 1 / REWARD 26 / Q_MAX  986141.6 / Loss  56502568.0
T 347674 / STATE explore / ε 0.626 / ACTION 5 / REWARD 26 / Q_MAX  1006651.7 / Loss  50505112.0
T 347675 / STATE explore / ε 0.626 / ACTI

T 347751 / STATE explore / ε 0.626 / ACTION 4 / REWARD 0 / Q_MAX  1024893.44 / Loss  74884864.0
T 347752 / STATE explore / ε 0.626 / ACTION 1 / REWARD 0 / Q_MAX  988621.25 / Loss  40349544.0
T 347753 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1010913.3 / Loss  71743552.0
T 347754 / STATE explore / ε 0.626 / ACTION 7 / REWARD 0 / Q_MAX  1007387.94 / Loss  64994624.0
T 347755 / STATE explore / ε 0.626 / ACTION 0 / REWARD 0 / Q_MAX  1022610.94 / Loss  44541656.0
T 347756 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  998492.3 / Loss  84034568.0
T 347757 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  881405.56 / Loss  68285512.0
T 347758 / STATE explore / ε 0.626 / ACTION 0 / REWARD 0 / Q_MAX  1026046.44 / Loss  130643872.0
T 347759 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  987544.3 / Loss  104625232.0
T 347760 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  885488.0 / Loss  92285280.0
T 347761 / STATE explore / ε 0.626 / ACTION 3 /

T 347838 / STATE explore / ε 0.626 / ACTION 3 / REWARD 0 / Q_MAX  959720.75 / Loss  43113624.0
T 347839 / STATE explore / ε 0.626 / ACTION 2 / REWARD 0 / Q_MAX  959694.6 / Loss  70686088.0
T 347840 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  950187.0 / Loss  51828760.0
T 347841 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1020448.0 / Loss  54758624.0
T 347842 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  981229.0 / Loss  75900952.0
T 347843 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  1019033.4 / Loss  77132128.0
T 347844 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  750788.2 / Loss  18895212.0
T 347845 / STATE explore / ε 0.626 / ACTION 5 / REWARD 0 / Q_MAX  854950.7 / Loss  44641224.0
T 347846 / STATE explore / ε 0.626 / ACTION 6 / REWARD 0 / Q_MAX  812451.25 / Loss  66937960.0
T 347847 / STATE explore / ε 0.626 / ACTION 3 / REWARD 0 / Q_MAX  781010.56 / Loss  53983696.0
T 347848 / STATE explore / ε 0.626 / ACTION 0 / REWARD 

...

T 383068 / STATE explore / ε 0.609 / ACTION 7 / REWARD 0 / Q_MAX  295275.56 / Loss  28837836.0
T 383069 / STATE explore / ε 0.609 / ACTION 2 / REWARD 0 / Q_MAX  278397.16 / Loss  41604704.0
T 383070 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  249571.94 / Loss  25269112.0
T 383071 / STATE explore / ε 0.609 / ACTION 5 / REWARD 0 / Q_MAX  246419.66 / Loss  30425684.0
T 383072 / STATE explore / ε 0.609 / ACTION 1 / REWARD 0 / Q_MAX  283586.88 / Loss  31784420.0
T 383073 / STATE explore / ε 0.609 / ACTION 3 / REWARD 0 / Q_MAX  293198.1 / Loss  11157328.0
T 383074 / STATE explore / ε 0.609 / ACTION 3 / REWARD 0 / Q_MAX  283191.72 / Loss  34044712.0
T 383075 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  279963.2 / Loss  49427088.0
T 383076 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  294411.53 / Loss  17922580.0
T 383077 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  282779.78 / Loss  20938004.0
T 383078 / STATE explore / ε 0.609 / ACTION 3 / REWA

T 383155 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  275059.88 / Loss  17308584.0
T 383156 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  270321.4 / Loss  20248538.0
T 383157 / STATE explore / ε 0.609 / ACTION 7 / REWARD 0 / Q_MAX  262197.3 / Loss  17258762.0
T 383158 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  265072.78 / Loss  51222944.0
T 383159 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  283878.72 / Loss  17562784.0
T 383160 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  222325.77 / Loss  23552822.0
T 383161 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  231246.6 / Loss  46870456.0
T 383162 / STATE explore / ε 0.609 / ACTION 5 / REWARD 0 / Q_MAX  273984.88 / Loss  26875920.0
T 383163 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  264383.6 / Loss  10354894.0
T 383164 / STATE explore / ε 0.609 / ACTION 6 / REWARD 0 / Q_MAX  271153.56 / Loss  18073566.0
T 383165 / STATE explore / ε 0.609 / ACTION 6 / REWARD

T 383242 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  245922.16 / Loss  17779214.0
T 383243 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  247704.97 / Loss  30666160.0
T 383244 / STATE explore / ε 0.608 / ACTION 4 / REWARD 0 / Q_MAX  253137.67 / Loss  21188752.0
T 383245 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  259614.61 / Loss  29354248.0
T 383246 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  258018.77 / Loss  28397208.0
T 383247 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  255016.66 / Loss  17327368.0
T 383248 / STATE explore / ε 0.608 / ACTION 1 / REWARD 0 / Q_MAX  236911.19 / Loss  29807632.0
T 383249 / STATE explore / ε 0.608 / ACTION 2 / REWARD 0 / Q_MAX  235342.45 / Loss  16972724.0
T 383250 / STATE explore / ε 0.608 / ACTION 2 / REWARD 0 / Q_MAX  270134.12 / Loss  20485024.0
T 383251 / STATE explore / ε 0.608 / ACTION 4 / REWARD 0 / Q_MAX  255752.73 / Loss  40366124.0
T 383252 / STATE explore / ε 0.608 / ACTION 6 / RE

T 383329 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  238899.03 / Loss  727231424.0
T 383330 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  245382.7 / Loss  21025958.0
T 383331 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  269926.56 / Loss  47669368.0
T 383332 / STATE explore / ε 0.608 / ACTION 2 / REWARD 0 / Q_MAX  278237.0 / Loss  26327616.0
T 383333 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  278893.94 / Loss  22736542.0
T 383334 / STATE explore / ε 0.608 / ACTION 0 / REWARD 0 / Q_MAX  242219.11 / Loss  10719983.0
T 383335 / STATE explore / ε 0.608 / ACTION 4 / REWARD 0 / Q_MAX  248078.86 / Loss  16018256.0
T 383336 / STATE explore / ε 0.608 / ACTION 2 / REWARD 0 / Q_MAX  247379.08 / Loss  23438408.0
T 383337 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  249468.88 / Loss  9795090.0
T 383338 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  245671.55 / Loss  7038741.5
T 383339 / STATE explore / ε 0.608 / ACTION 6 / REWAR

T 383416 / STATE explore / ε 0.608 / ACTION 0 / REWARD 0 / Q_MAX  266762.16 / Loss  17016708.0
T 383417 / STATE explore / ε 0.608 / ACTION 7 / REWARD 0 / Q_MAX  246270.06 / Loss  16615056.0
T 383418 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  243448.89 / Loss  15772057.0
T 383419 / STATE explore / ε 0.608 / ACTION 5 / REWARD 0 / Q_MAX  256186.77 / Loss  18946496.0
T 383420 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  248896.8 / Loss  11651092.0
T 383421 / STATE explore / ε 0.608 / ACTION 4 / REWARD 0 / Q_MAX  236417.42 / Loss  13581822.0
T 383422 / STATE explore / ε 0.608 / ACTION 1 / REWARD 0 / Q_MAX  254733.4 / Loss  20538086.0
T 383423 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  230651.73 / Loss  28841554.0
T 383424 / STATE explore / ε 0.608 / ACTION 7 / REWARD 0 / Q_MAX  235131.73 / Loss  27877092.0
T 383425 / STATE explore / ε 0.608 / ACTION 6 / REWARD 0 / Q_MAX  243669.12 / Loss  17125486.0
T 383426 / STATE explore / ε 0.608 / ACTION 7 / REWA